In [0]:
# genesis_workbench.dev_mmt_core_test

In [0]:
catalog_name = "genesis_workbench"
schema_name = "dev_mmt_core_test"

In [0]:
# Construct the SQL query string
drop_schema_query = f"""
DROP SCHEMA IF EXISTS {catalog_name}.{schema_name} CASCADE;
"""

# create_schema_query = f"""
# CREATE SCHEMA {catalog_name}.{schema_name};
# """

# Execute the SQL queries
spark.sql(drop_schema_query)
# spark.sql(create_schema_query)

In [0]:
# full_schema = f"{catalog_name}.{schema_name}"
# tables_df = spark.sql(f"SHOW TABLES IN {full_schema}")
# tables = tables_df.collect()

In [0]:
# display(tables)

In [0]:
import os

# Set these environment variables
os.environ['DATABRICKS_HOST'] = 'https://adb-830292400663869.9.azuredatabricks.net/'
os.environ['DATABRICKS_TOKEN'] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

In [0]:
from databricks.sdk import WorkspaceClient

def delete_serving_endpoints_with_prefix(prefix):
    w = WorkspaceClient()

    # List all serving endpoints
    response = w.serving_endpoints.list()
    endpoints = response.endpoints  # Access the list of endpoints

    # Filter endpoints with the specified prefix
    endpoints_to_delete = [
        ep for ep in endpoints 
        if ep.name.startswith(prefix)
    ]

    if not endpoints_to_delete:
        print(f"No serving endpoints found with prefix '{prefix}'")
        return

    print(f"Found {len(endpoints_to_delete)} serving endpoints with prefix '{prefix}':")
    for ep in endpoints_to_delete:
        print(f"- {ep.name}")

    # Confirm before proceeding
    confirm = input(f"Delete all serving endpoints with prefix '{prefix}'? (yes/no): ")
    if confirm.lower() != 'yes':
        print("Deletion cancelled.")
        return

    for ep in endpoints_to_delete:
        try:
            w.serving_endpoints.delete(name=ep.name)
            print(f"✓ Deleted serving endpoint: {ep.name}")
        except Exception as e:
            print(f"✗ Error deleting {ep.name}: {str(e)}")

# Usage
# delete_serving_endpoints_with_prefix("gwb_scimilarity_")

In [0]:
delete_serving_endpoints_with_prefix("gwb_scimilarity_")

In [0]:
import mlflow

def delete_models_by_prefix_or_suffix(prefix=None, suffix=None):
    # Search for all registered models
    models = mlflow.search_registered_models()
    
    for model in models:
        model_name = model.name
        
        # Check if the model name matches the prefix or suffix
        if (prefix and model_name.startswith(prefix)) or (suffix and model_name.endswith(suffix)):
            # Delete all versions of the model
            for version in model.latest_versions:
                mlflow.delete_model_version(name=model_name, version=version.version)
            
            # Delete the registered model
            mlflow.delete_registered_model(name=model_name)

# Example usage
delete_models_by_prefix_or_suffix(prefix="gwb_scimilarity_")